# Select GPU

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


# Import Module

In [2]:
import numpy as np
import pandas as pd
import math
import keras
import csv
import os
import random
import time
import pickle
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input,Conv1D, Dense, MaxPool1D, Activation, AvgPool1D, GlobalAveragePooling1D
from keras.layers import Flatten, Add, Concatenate, Dropout, BatchNormalization, merge
from keras import regularizers, initializers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from scipy.stats.stats import pearsonr
from keras.optimizers import Adam
from keras import regularizers
from keras import backend as K

Using TensorFlow backend.


In [3]:
multi_CNN = True
activation = 'relu' # selu
use_meta = True

# Import Data

In [4]:
'''read signal'''
file_path = '/data/put_data/timliu/BG/'
if multi_CNN:
    with open(file_path+'train_test/data/0918/dict_X_C10_S5_50.0.pickle', 'rb') as handle:
        X50 = pickle.load(handle)
    with open(file_path+'train_test/data/0918/dict_X_C10_S5_100.0.pickle', 'rb') as handle: # 100
        X5 = pickle.load(handle) 
    with open(file_path+'train_test/data/0918/dict_X_C10_S5_200.0.pickle', 'rb') as handle: 
        X10 = pickle.load(handle) 
    with open(file_path+'train_test/data/0918/dict_X_C10_S5_25.0.pickle', 'rb') as handle: 
        X25 = pickle.load(handle) 
else:
    with open('train_test/data/0918/dict_X_C10_S5_200.0.pickle', 'rb') as handle:
        X = pickle.load(handle)
'''read meta'''
with open(file_path+'train_test/dict_meta3_0901.pickle', 'rb') as handle:
    meta = pickle.load(handle)

# Parameters

In [5]:
# data split
split_ratio = 0.8
# model parameter
l_2 = 1e-6
dropout_rate = 0.4
lr_rate = 0.001

# Embedding
n_sample = 6000
n_meta = len(meta['003_1'])
n_channel = 10

# Convolution
kernel_size = 15
local_filters = 16
local_kernel_size = 3
local_pool_size = 2
filters = 64
filter_size = 15
strides = 1
depth = 29

# Training
batch_size = 64
n_batch_per_epoch = 200
epochs = 50
kernel_initializer = 'he_normal'

d = pd.read_csv(file_path + "train_test/data/new_IRB_summary_804.5.csv",dtype="str")
print(d[:5])

  Person No Age Gender Height  SYS  DIA   HR    G  PWV   BMI  ...      f18  \
0         3  53      0    157  143  104   73  106  6.5  25.4  ...    0.001   
1         3  53      0    157  144   95   73   95  6.5  25.4  ...   0.0013   
2         4  49      0    150  117   85   75   99    7  23.4  ...    8e-04   
3         4  49      0    150  133   92   80   94    7  23.4  ...    0.001   
4         5  55      1    167  142   79  104   88    7  24.4  ...   0.0014   

      f19     f20     f21     f22     f23          f24     ID      Date  Time  
0  353268  0.8983  0.0049  0.0017  131210  0.810468912  003_1  20170302  0935  
1  356900  0.9129  0.0053  0.0027  119821  0.820373018  003_2  20170302  0939  
2  657728  0.8002  0.0024  0.0011  240208  0.741961393  004_1  20170303  0817  
3  447514  0.7975   0.003  0.0025  156888  0.721089641  004_2  20170303  0827  
4  380280  0.5866  0.0015  0.0021  130575  0.558661023  005_1  20170303  0837  

[5 rows x 48 columns]


# Splitting Data

In [6]:
'''split training and testing data set'''
D_id = list(set([row['Person No'] for index, row in d.iterrows() if float(row['G']) >= 200]))
np.random.shuffle(D_id)
ND_id = np.asarray(d.loc[np.asarray(d['G'], dtype = 'float64') < 200]['Person No'].unique())
np.random.shuffle(ND_id)

train_id = np.hstack((ND_id[:int(len(ND_id) * split_ratio)], D_id[:int(len(D_id) * split_ratio)]))
# train_id = np.hstack((ND_id[:int(len(ND_id) * split_ratio)]))
train_id

test_id = np.hstack((ND_id[int(len(ND_id) * split_ratio):], D_id[int(len(D_id) * split_ratio):]))
# test_id = np.hstack((ND_id[int(len(ND_id) * split_ratio):]))
test_id

test_ind = [index for index, row in d.iterrows() if np.any(test_id == row['Person No'])] 
train_ind = [index for index, row in d.iterrows() if np.any(train_id == row['Person No'])]   

train_file = np.array(d['ID'][train_ind])
test_file = np.array(d['ID'][test_ind])
lab = np.array(d['G']).astype('float32')
train_lab = np.array(lab[train_ind])
train_lab = train_lab.astype('float32')
test_lab  = np.array(lab[test_ind])
test_lab  = test_lab.astype('float32')

In [7]:
def train_generator(batch_size):
    if multi_CNN:
        while 1:
            X5_ = []
            X10_ = []
            X25_ = []
            X50_ = []
            M_ = []
            Y_ = []
            for _ in range(batch_size):
                # randomly select a record
                i = random.randint(0,len(train_file)-1)
                x = X5[train_file[i]] 
                x10 = X10[train_file[i]]
                x25 = X25[train_file[i]]
                x50 = X50[train_file[i]]
                x2 = meta[train_file[i]]
                # random select a subrecord from 0~4
                j = random.randint(0, 4)
                x1 = x[j]
                x101 = x10[j]
                x251 = x25[j]
                x501 = x50[j]
                # append on list
                X5_.append(x1)
                X10_.append(x101)
                X25_.append(x251)
                X50_.append(x501)
                M_.append(x2)
                Y_.append(train_lab[i])
            # stack together batch size numbers of training data    
            X5_ = np.asarray(X5_)
            X10_ = np.asarray(X10_)
            X25_ = np.asarray(X25_)
            X50_ = np.asarray(X50_)
            M_ = np.asarray(M_)
            if use_meta:
                COM_ = [X5_,X10_,X25_,X50_,M_]
            else:
                COM_ = [X5_,X10_,X25_,X50_]
            Y_ = np.asarray(Y_)
            yield COM_, Y_
    else:    
        while 1:
            X_ = []
            M_ = []
            Y_ = []
            for _ in batch_size:
                id_ = train_file[random.randint(0, len(train_file) - 1)] # random select a sample
                signal_id = random.randint(0, 4) # random select a signal
                x = X[id_][signal_id]
                X_.append(x)
                M_.append(meta[id_])
                Y_.append(train_lab[id_])
            X_ = np.asarray(X_)
            M_ = np.asarray(M_)
            Y_ = np.asarray(Y_)
            if use_meta:
                yield [X_, M_], Y_
            else:
                yield [X_], Y_

In [8]:
def validation_generation(test_file,test_lab):
    if multi_CNN:
        X5_ = []
        X10_ = []
        X25_ = []
        X50_ = []
        M_ = []
        Y_ = []
        for i in range(len(test_file)):
            x = X5[test_file[i]] 
            x10 = X10[test_file[i]]
            x25 = X25[test_file[i]]
            x50 = X50[test_file[i]]
            x2 = meta[test_file[i]]
            for j in range(5):
                X5_.append(x[j])
                X10_.append(x10[j])
                X25_.append(x25[j])
                X50_.append(x50[j])
                M_.append(x2)
                Y_.append(test_lab[i])
        X5_ = np.asarray(X5_)
        X10_ = np.asarray(X10_)
        X25_ = np.asarray(X25_)
        X50_ = np.asarray(X50_)
        M_ = np.asarray(M_)
    #     COM_ = [X5_,X10_,X25_,X50_,M_]
        Y_ = np.asarray(Y_)
        print("# records = ", len(Y_))
        if use_meta:
            return X5_, X10_,X25_,X50_,M_, Y_ 
        else:
            return X5_, X10_,X25_,X50_, Y_
    else:
        X_ = []
        M_ = []
        Y_ = []
        for i in range(len(test_file)):
            id_ = test_file[i]
            for j in range(5):
                signal_id = j # random select a signal
                x = X[id_][signal_id]
                X_.append(x)
                M_.append(meta[id_])
                Y_.append(test_lab[id_])
        X_ = np.asarray(X_)
        M_ = np.asarray(M_)
        Y_ = np.asarray(Y_)
        print("# records = ", len(Y_))
        if use_meta:
            return X_, M_, Y_
        else:
            return X_, Y_

In [9]:
def one_train_set(size = 1610):
    if multi_CNN:
        X5_ = []
        X10_ = []
        X25_ = []
        X50_ = []
        M_ = []
        Y_ = []
        count = 0
        while count < size:
            count = count + 1
            # select a random sample
            i = random.randint(0,len(train_file)-1)
            x = X5[train_file[i]] 
            x10 = X10[train_file[i]]
            x25 = X25[train_file[i]]
            x50 = X50[train_file[i]]
            x2 = meta[train_file[i]]
            j = random.randint(0, 4)
            x1 = x[j]
            x101 = x10[j]
            x251 = x25[j]
            x501 = x50[j]
            X5_.append(x1)
            X10_.append(x101)
            X25_.append(x251)
            X50_.append(x501)
            M_.append(x2)
            Y_.append(train_lab[i])
        X5_ = np.asarray(X5_)
        X10_ = np.asarray(X10_)
        X25_ = np.asarray(X25_)
        X50_ = np.asarray(X50_)
    #     COM_ = [X5_,X10_,X25_,X50_,M_]
        M_ = np.asarray(M_)
        Y_ = np.asarray(Y_) 
        if use_meta:
            return X5_, X10_,X25_,X50_,M_, Y_
        else:
            return X5_, X10_,X25_,X50_, Y_
    else:
        X_ = []
        M_ = []
        Y_ = []
        count = 0
        while count < size:
            id_ = train_file[i]
            signal_id = random.randint(0,len(train_file)-1) # random select a signal
            x = X[id_][signal_id]
            X_.append(x)
            M_.append(meta[id_])
            Y_.append(train_lab[id_])
            count += 1
        X_ = np.asarray(X_)
        M_ = np.asarray(M_)
        Y_ = np.asarray(Y_)
        if use_meta:
            return X_, M_, Y_
        else:
            return X_,Y_

In [10]:
def local_cnn(filters,kernel_size,strides,pool_size,inputs):
    if activation == 'selu':
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inputs)
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inp)
        inp = MaxPool1D(pool_size = local_pool_size, padding = 'same')(inp)
        inp = Activation(activation='selu')(inp)
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inp)
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inp)
    else:
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inputs)
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inp)
        inp = MaxPool1D(pool_size = local_pool_size, padding = 'same')(inp)
        inp = BatchNormalization()(inp)
        inp = Activation(activation=activation)(inp)
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inp)
        inp = Conv1D(filters = local_filters, kernel_size = local_kernel_size, strides = strides, padding = 'same',kernel_regularizer=regularizers.l2(l_2))(inp)
    return inp

# Model Function

In [11]:
def conv_block(ip, nb_filter, filter_size, dropout_rate = None):
    x = BatchNormalization()(ip)
    x = Activation('relu')(x)
    x = Conv1D(nb_filter, filter_size, strides = 1, kernel_initializer = kernel_initializer, padding = "same",kernel_regularizer=regularizers.l2(l_2))(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    return x

def transition_block(ip, nb_filter, filter_size, dropout_rate=None):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1
    
    x = BatchNormalization()(ip)
    x = Conv1D(nb_filter, filter_size, strides = 1, kernel_initializer = kernel_initializer, padding = "same",kernel_regularizer=regularizers.l2(l_2))(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    x = AvgPool1D(pool_size = 2, strides = 2)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, filter_size, dropout_rate)
        feature_list.append(x)
        x = merge(feature_list, mode='concat', concat_axis=concat_axis)
        nb_filter += growth_rate

    return x, nb_filter

def create_dense_net(depth=13, nb_dense_block=3, growth_rate=12, nb_filter=32, dropout_rate=None,verbose=True):
    sig_inp5 = Input(shape = (3000, 10))
    inp5 = local_cnn(local_filters,local_kernel_size,strides,local_pool_size,sig_inp5)
    
    sig_inp10 = Input(shape = (6000, 10))
    inp10 = local_cnn(local_filters, local_kernel_size, strides, local_pool_size, sig_inp10)

    sig_inp25 = Input(shape = (750, 10))
    inp25 = local_cnn(local_filters, local_kernel_size, strides, local_pool_size, sig_inp25)

    sig_inp50 = Input(shape = (1500, 10))
    inp50 = local_cnn(local_filters, local_kernel_size, strides, local_pool_size, sig_inp50)
    
    inp_con = keras.layers.concatenate([inp5, inp10, inp25, inp50], axis = 1)
    metai = Input(shape = (n_meta, ))
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - (nb_dense_block+1)) % nb_dense_block == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - (nb_dense_block+1)) / nb_dense_block)

    # Initial convolution
    x = Conv1D(nb_filter, filter_size, strides = 1, kernel_initializer=kernel_initializer, padding="same", name="initial_conv2D",kernel_regularizer=regularizers.l2(l_2))(inp_con)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate)
        # add transition_block
        x = transition_block(x, nb_filter, filter_size, dropout_rate=dropout_rate)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate)

    x = GlobalAveragePooling1D()(x)
#     x = BatchNormalization()(x)
#     x = Activation(activation = 'relu')(x)
#     x = keras.layers.concatenate([x, metai])
#     x = Dense(256, activation='relu',kernel_regularizer=regularizers.l2(l_2),kernel_initializer=kernel_initializer)(x)
#     x = Dropout(dropout_rate)(x)
#     x = Dense(256, activation='relu',kernel_regularizer=regularizers.l2(l_2),kernel_initializer=kernel_initializer)(x)
#     x = Dropout(dropout_rate)(x)
#     x = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(l_2),kernel_initializer=kernel_initializer)(x)
#     x = Dropout(dropout_rate)(x)
#     x = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(l_2),kernel_initializer=kernel_initializer)(x)
#     x = Dropout(dropout_rate)(x)
#     x = Dense(64, activation='relu',kernel_regularizer=regularizers.l2(l_2),kernel_initializer=kernel_initializer)(x)
#     x = Dropout(dropout_rate)(x)
#     x = Dense(64, activation='relu',kernel_regularizer=regularizers.l2(l_2),kernel_initializer=kernel_initializer)(x)
#     x = Dropout(dropout_rate)(x)
#     x = Dense(1, activation='linear',kernel_regularizer=regularizers.l2(l_2),kernel_initializer=kernel_initializer)(x)
    
    new_inp = keras.layers.concatenate([x,metai])
    new_inp2 = x
    new_inp = BatchNormalization()(new_inp)
    new_inp = Activation(activation=activation)(new_inp)
    #             if use_meta:
    #                 new_inp = keras.layers.concatenate([new_inp,meta_inp])
    new_inp = Dense(128,kernel_regularizer=regularizers.l2(l_2),kernel_initializer = kernel_initializer)(new_inp)
    new_inp = Dropout(dropout_rate)(new_inp)
    new_inp = keras.layers.concatenate([new_inp, new_inp2])
    new_inp = BatchNormalization()(new_inp)
    new_inp = Activation(activation=activation)(new_inp)
    new_inp = Dense(64,kernel_regularizer=regularizers.l2(l_2),kernel_initializer = kernel_initializer)(new_inp)
    new_inp = Dropout(dropout_rate)(new_inp)
    out = Dense(1,activation='linear', kernel_regularizer=regularizers.l2(l_2))(new_inp)

    densenet = Model(inputs = [sig_inp5, sig_inp10, sig_inp25, sig_inp50, metai], output = out, name = "create_dense_net")

    if verbose: print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

# Model Compile

In [12]:
model = create_dense_net(depth=depth, nb_dense_block=4, growth_rate=12, nb_filter=32, dropout_rate=0.3,verbose=True)
adam = Adam(lr=lr_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay = 0.0)
model.compile(loss='mae', optimizer=adam)
model.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


DenseNet-29-12 created.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3000, 10)      0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 6000, 10)      0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 750, 10)       0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 1500, 10)      0                                            
___________________________________________________________________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:99: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso..., name="create_dense_net")`


# Directory Name

In [13]:
'''model name'''
time_str = time.strftime("%m%d")
drop = str(math.ceil(dropout_rate/0.1))
model_name = time_str + '_D' + drop + '_l' + str(l_2) + '_C10_MCNN_Dense_'+activation+'_L'+str(depth)+'_Meta=' + str(use_meta) +'_idenFC'
directory = 'train_' + time_str + '/' + model_name + '/' 

'''model training'''
print('Model will be saved to ', directory, '...........')

if not os.path.exists(directory):
    os.makedirs(directory)

Model will be saved to  train_0924/0924_D4_l1e-06_C10_MCNN_Dense_relu_L29_Meta=True_idenFC/ ...........


# Model Tools

In [14]:
checkpoint = ModelCheckpoint(directory + "model.h5",
                    monitor='val_loss',
                    verbose=1,
                    save_best_only=True,
                    mode='auto')

class LossHistory(Callback):
    def on_train_begin(self,logs={}):
        self.loss=[]
        self.val_loss=[]
    def on_epoch_end(self,epoch,logs={}):
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

loss_history = LossHistory()

earlystopping = EarlyStopping(monitor='val_loss', patience = 30, mode = 'auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience = 5, min_lr = 0, cooldown = 5, verbose = True)

# Model Training

In [15]:
Val_X5, Val_X10, Val_X25, Val_X50, Val_M,Val_Y = validation_generation(test_file,test_lab)
model.fit_generator(generator=train_generator(batch_size),
                validation_data=([Val_X5, Val_X10, Val_X25,Val_X50,Val_M],Val_Y),
                steps_per_epoch = n_batch_per_epoch,
                epochs=epochs,
                callbacks = [loss_history,checkpoint,earlystopping,reduce_lr])

# records =  1610
Epoch 1/50
200/200 [==============================] - 217s - loss: 23.0287 - val_loss: 19.4010
Epoch 2/50
200/200 [==============================] - 164s - loss: 12.8009 - val_loss: 18.2768
Epoch 3/50
200/200 [==============================] - 164s - loss: 12.7481 - val_loss: 19.8612
Epoch 4/50
200/200 [==============================] - 165s - loss: 12.1954 - val_loss: 14.9912
Epoch 5/50
200/200 [==============================] - 165s - loss: 12.0309 - val_loss: 12.9688
Epoch 6/50
200/200 [==============================] - 165s - loss: 12.0201 - val_loss: 12.7526
Epoch 7/50
200/200 [==============================] - 164s - loss: 11.9140 - val_loss: 14.4457
Epoch 8/50
200/200 [==============================] - 165s - loss: 11.5929 - val_loss: 13.3992
Epoch 9/50
200/200 [==============================] - 164s - loss: 11.9075 - val_loss: 13.8148
Epoch 10/50
200/200 [==============================] - 164s - loss: 11.6736 - val_loss: 14.3834
Epoch 11/50
200/200 [==========

200/200 [==============================] - 162s - loss: 9.7769 - val_loss: 17.0344
Epoch 38/50
200/200 [==============================] - 162s - loss: 9.5779 - val_loss: 17.2038
Epoch 39/50
200/200 [==============================] - 162s - loss: 9.6492 - val_loss: 17.1398
Epoch 40/50
200/200 [==============================] - 162s - loss: 9.5801 - val_loss: 17.1185
Epoch 41/50
200/200 [==============================] - 162s - loss: 9.6473 - val_loss: 17.1932
Epoch 42/50
200/200 [==============================] - 162s - loss: 9.7810 - val_loss: 17.1189


In [16]:
'''loss history fig'''
matplotlib.use('Agg')
sk = 0
plt.figure(0)
plt.plot(range(len(loss_history.loss[sk:])),loss_history.loss[sk:],label='train_loss')
plt.plot(range(len(loss_history.val_loss[sk:])),loss_history.val_loss[sk:],label='val_loss')
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.ylim([0, max(loss_history.val_loss+loss_history.loss)])
plt.savefig(directory + "loss_history.png", dpi = 300)
plt.close()

'''plot train/test mae/cor fig'''
tmp = [model_name]

# recording
tmp.append(loss_history.loss[loss_history.val_loss.index(min(loss_history.val_loss))])
tmp.append(min(loss_history.val_loss))
tmp.append(loss_history.loss[-1])
tmp.append(loss_history.val_loss[-1])

model_test = load_model(directory + "model.h5")

# plot training
if use_meta:
    train_X1, train_X2, train_X3, train_X4, train_M,train_Y1 = one_train_set()
    Y_pred = model_test.predict([train_X1, train_X2, train_X3, train_X4,train_M])
else:
    train_X1, train_X2, train_X3, train_X4,train_Y1 = one_train_set()
    Y_pred = model_test.predict([train_X1, train_X2, train_X3, train_X4])
Y_pred = np.hstack(Y_pred)

cor = pearsonr(train_Y1,Y_pred)[0]
mae = mean_absolute_error(train_Y1,Y_pred)

# recording
tmp.append(mae)
tmp.append(cor)


f = pd.DataFrame({'y_true': train_Y1,'y_pred': Y_pred})
f.to_csv(directory + "pred_train.csv")


plt.figure(0)
plt.scatter(train_Y1, Y_pred, alpha = .15, s = 20)
plt.xlabel('True_Y')
plt.ylabel('Pred_Y')
plt.title("Training data \n" + "MAE = %4f; Cor = %4f; #samples = %d" % (mae,cor, len(train_Y1)))
plt.savefig(directory+"plot_scatter_train.png", dpi = 300)
plt.close()

# plot testing
if use_meta:
    Y_pred = model_test.predict([Val_X5, Val_X10, Val_X25, Val_X50, Val_M])
else:
    Y_pred = model_test.predict([Val_X5, Val_X10, Val_X25, Val_X50])
    
Y_pred = np.hstack(Y_pred)

f = pd.DataFrame({'y_true': Val_Y,'y_pred': Y_pred})
f.to_csv(directory + "pred_test.csv")

cor = pearsonr(Val_Y,Y_pred)[0]
mae = mean_absolute_error(Val_Y,Y_pred)

# recording
tmp.append(mae)
tmp.append(cor)

'''scatter plot for test data'''
plt.figure(0)
plt.scatter(Val_Y, Y_pred, alpha = .15, s = 20)
plt.xlabel('True_Y')
plt.ylabel('Pred_Y')
plt.title("Testing data \n" + "MAE = %4f; Cor = %4f; #samples = %d" % (mae, cor, len(Val_Y)))
plt.savefig(directory + "plot_scatter_test.png", dpi = 300)
plt.close()

'''save model detail'''
f = pd.DataFrame([tmp], columns = ['model_name', 'train_loss', 'val_loss', 'final_train_loss', 'final_val_loss', 'train_mae', 'train_cor', 'test_mae', 'test_cor'])
f.to_csv(directory + 'model_detail.csv')

'''save test ID'''
f2 = pd.DataFrame([ID for ID in test_file], columns = ['test_ID'])
f2.to_csv(directory + 'test_id.csv')

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1242: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


# Clear GPU Memory

In [17]:
'''
%%javascript
Jupyter.notebook.session.delete();
'''

'\n%%javascript\nJupyter.notebook.session.delete();\n'